In [64]:
import os
import pdfplumber
import pandas as pd
from scipy.spatial.distance import cdist
import promptlib

In [65]:
#Abre o diretório onde estão os laudos a serem lidos
prompter = promptlib.Files()
dir = prompter.dir()
print(dir)

                

D:\OneDrive - incubate web-enabled systems\house-pricing\le-pdf\2019-2020-2021


In [66]:
#Mostra os laudos contidos no diretório
files=os.listdir(dir)


In [67]:
#Conta a quantidade laudos 
n_laudos=len(files)
n_laudos

540

In [68]:

#Lista com nome dos campos que serão armazenados
_os=[]
_end=[]
_andar=[]
_tipo = []
_ofer_trans=[]
_municipio=[]
_bairro=[]
_lat,_long,_cep=[],[],[]
_idade=[]
_vagas=[]
_pad=[]
_cons=[]
_banheiros=[]
_quartos=[]
_equip=[]
_area=[]
_valor_unit=[]
_valor=[]
_condo=[]
_data=[]
_empresa=[]



In [69]:
#Função que Junta o laudo em uma lista de linhas
def carrega_laudos():
    global laudo
    laudo=files[n]                                
    _os.append(laudo)                             #Carrega o laudo n da lista de laudos
    print("laudo:",n,laudo)
    pag = [[] for _ in range(4)]                  #Cria uma lista vazia com as páginas
    #print(dir+'/'+laudo)
    with pdfplumber.open(dir+'/'+laudo) as pdf:   #Abre o laudo com o pdfplumber
        linhas=[]                                 #Cria uma lista vazia para as linhas
        for pg in range(0,4):                     #Faz um loop com o n. de paginas
            page = pdf.pages[pg]                  #Lê uma página
            page_text = page.extract_text()       #Extrai o texto da página
            lines = page_text.splitlines()        #Extrai as linhas
            linhas.append(lines)                  #Adiciona as linhas na lista linhas
    return linhas                                 #Retorna as linhas geradas

In [71]:
for n in range(n_laudos):
    laudos = carrega_laudos()                 #Carrega paginas do laudo
    #print("N. de páginas carregadas:",len(laudos))
    for p in range(len(laudos)):              #Loop em todas as paginas
        #print("Pagina lida:",p)
        pag=laudos[p]
        for (index,item) in enumerate(pag):
            #print(index,item)

            if item == 'Categoria UF Município do Imóvel':
                municipio = pag[index+1].split(" ")[2]
                _municipio.append(municipio)
                tipo = pag[index+1].split(" ")[0]
                if tipo == "Casa": tipo=1
                else: tipo=2
                _tipo.append(tipo)
            if item == 'Endereço do Imóvel':
                end=pag[index+1]
                if tipo==2:
                    try:
                        i = 1
                        while i < 20:
                              temp = end.split(' ')
                              unid = temp[i]
                              #print("Temp:",temp,"Unid:",unid)
                              if unid=="Unidade":
                                    andar=temp[i+1].split(",")[0]
                                    if (andar[0]==1) or (andar[0]==2) or (andar[0]==3):
                                        if len(andar)>3:
                                            andar = andar[0:2]
                                        else: andar = andar[0:1]
                                    else:andar=andar[0:1]
                                    break
                              i += 1
                    except:andar=1
                else: 
                    andar=0
                #print("Andar Final:",andar)
                _andar.append(andar)
                _end.append(end)
                    
            

            if item =="Hemisfério Graus Min Seg Graus Min Seg Datum CEP":
                lat_grau = int(pag[index+1].split(' ')[1].split('º')[0])
                lat_min  = int(pag[index+1].split(' ')[2].split("'")[0])/60
                lat_seg  = float(pag[index+1].split(' ')[3].split('"')[0].replace(',','.'))/3600
                long_grau=int(pag[index+1].split(' ')[4].split('º')[0])
                long_min=int(pag[index+1].split(' ')[5].split("'")[0])/60
                long_seg=float(pag[index+1].split(' ')[6].split('"')[0].replace(',','.'))/3600
                lat=-(lat_grau+lat_min+lat_seg)
                _lat.append(lat)
                long=-(long_grau+long_min+long_seg)
                _long.append(long)
                cep = pag[index+1].split(' ')[-1]
                _cep.append(cep)
                
            if item == 'Idade Estimada (Anos)':
                    idade=int(pag[index+2].split(' ')[-1])
                    idade=(idade+(idade-5))/2
                    #print(idade)                    
                    _idade.append(idade)
            if item =='Ocupação Térreo Estado Conservação Idade Estimada (Anos)':
                    indice=index
                    idade=int(pag[index+1].split(' ')[-1])
                    idade=(idade+(idade-5))/2
                    #print('Idade=',idade)
                    _idade.append(idade)  
                    

            if item == 'Terreno Total Cobertas Total Descobertas Total (R$)':
                vagas = pag[index+1].split(' ')[-2]
                if len(vagas)>4: vagas=pag[index+1].split(" ")[-3]
                vagas=int(float(vagas.replace(",",".")))
                #print("Tipo:",tipo," vagas:",vagas)  
                _vagas.append(vagas)      
                
            if item == 'Total Cobertas Bloqueadas/Bloqueantes Total Descobertas':
                vagas = pag[index+3].split(' ')[-1]
                if vagas=="Interna":
                    vagas=pag[index+2].split(' ')[-1]
                vagas=int(float(vagas.replace(",",".")))
                #print("Tipo:",tipo," vagas:",vagas)                
                _vagas.append(vagas)
            
            
            if item == 'Privativa da Unidade Privativa Descontínua Comuns Total Construida':
                area=float(pag[index+1].split(' ')[-1].replace(',','.'))
                _area.append(area)
                
                
            if item == 'da Unidade Descontínua Averbada da Unidade Descontínua Averbada da Unidade Descontínua Construída':
                area=float(pag[index+1].split(' ')[-1].replace(',','.'))
                _area.append(area)
                

   #PADRÃO DE ACABAMENTO
            x1='Paredes Acabamentos Padrão de Acabamento Estado de Conservação'
            x2='Compartimentos com Armários Salões de Lazer Padrão de Acabamento Estado de Conservação'
            if item == x1:
                #print('x1==>',x1)
                temp=pag[index+2]
                if len(temp.split(' '))<7:
                    pad=1
                else:
                    temp=temp.split('(')
                    pad=temp[0].split(' ')[-2]
                    pad1=temp[1].split(' ')[:-2]
                    pad1=" ".join(pad1)
                    pad=pad+' '+pad1
                if pad=='Normal c/ aspectos de baixo)':
                    pad=3
                elif pad=='Normal forte predominância)':
                    pad=4
                elif pad=='Normal c/ aspectos de alto)':
                    pad=5
                elif pad=='Alto por predominância)':
                    pad=6
                elif pad=='Alto superior, luxo)':
                    pad=7
                else: pad=4
                _pad.append(pad)
            elif item == x2:
                #print('x2==>',x2)
                temp=pag[index+1]
                if len(temp.split(' '))<7:
                    pad=1
                else:
                    temp=temp.split('(')               
                    pad=(temp[0]+temp[1]).split(' ')[2:-2]
                    pad=" ".join(pad)
                if pad=='Normal c/ aspectos de baixo)':
                    pad=3
                elif pad=='Normal forte predominância)':
                    pad=4
                elif pad=='Normal c/ aspectos de alto)':
                    pad=5
                elif pad=='Alto por predominância)':
                    pad=6
                elif pad=='Alto superior, luxo)':
                    pad=7
                else: pad=4
                _pad.append(pad)

            #CONSERVAÇÃO
            x1='Paredes Acabamentos Padrão de Acabamento Estado de Conservação'
            x2='Compartimentos com Armários Salões de Lazer Padrão de Acabamento Estado de Conservação'
            if item == x1:
                #print('x1==>',x1)
                temp=pag[index+2]
                if len(temp.split(' '))<7:
                    cons=1
                else:
                    temp=temp.split(')')
                    cons=(temp[-2]+temp[-1]).strip()
                if (cons == 'Regular (reparos simples') or (cons == 'Regular reparos simples'):
                    cons=3
                elif (cons == 'Regular (reparos importantes') or (cons == 'Regular reparos importantes'):
                    cons=2
                elif (cons == 'Bom (aparência de usado') or (cons == 'Bom aparência de usado'):
                    cons=4
                elif (cons == 'Bom (aparência de novo') or (cons == 'Bom aparência de novo'):
                    cons=4
                else: cons=5
                #print('Cons1=>',cons)
                _cons.append(cons)
            elif item == x2:
                #print('x2==>',x2)
                temp=pag[index+1]
                if len(temp.split(' '))<7:
                    cons=1
                else:
                    temp=temp.split('(')
                    cons=(temp[-2]+temp[-1]).split(')')[-2].strip()
                if (cons == 'Regular (reparos simples') or (cons == 'Regular reparos simples'):
                    cons=3
                elif (cons == 'Regular (reparos importantes') or (cons == 'Regular reparos importantes'):
                    cons=2
                elif (cons == 'Bom (aparência de usado') or (cons == 'Bom aparência de usado'):
                    cons=4
                elif (cons == 'Bom (aparência de novo') or (cons == 'Bom aparência de novo'):
                    cons=4
                else: cons=5
                #print('Cons1=>',cons)
                _cons.append(cons)
            
            
            if item == 'Divisão Interna':
                #print('x1==>',x1)
                divisao=pag[index+1].split(' ')
                if  not 'Quartos,' in divisao:
                    try:
                        divisao1=pag[index+2].split(' ')
                        divisao=divisao+divisao1
                    except:
                        _quartos.append(1)
                        _banheiros.append(1)
                        break
                for item in divisao:
                    banheiros=0
                    for prev, current, nxt in zip(divisao, divisao[1:], divisao[2:]):
                        if current == 'Quartos,':
                            quartos = prev
                        if (current=="Banheiro") and (nxt =="Privativo,") :
                            banheiros=int(prev)
                        if (current=="Banheiro") and (nxt =="Privativos,"):
                            banheiros=banheiros+int(prev)
                        if (current=="Banheiro") and (nxt =="Social,"):
                            banheiros=banheiros+int(prev)
                quartos=int(quartos)
                _quartos.append(quartos)
                _banheiros.append(banheiros)
            x='Equipamentos da Unidade ou Disponíveis para a Unidade no Empreendimento'           
            if (item == x):
                equip=pag[index+1].split(',')
                if equip =='Não Possui Equipamentos.':
                    equip=0
                else: 
                    equip = len(equip)
                if equip ==1: equip = 0
                _equip.append(equip)
 

            x='Empresa CNPJ Representante Legal CPF'          
            if (item == x):
                empresa=pag[index+1]
                #print (empresa)
                _empresa.append(empresa)
                
            x='Empreendimento/condomínio'       
            if (item == x):
                temp=pag[index+2].split(' ')[0]
                if temp=="Infra-Estrutura": condo=0
                else:condo=1
                _condo.append(condo)                    
                #print("Condo:",temp," ",condo) 
            if item=='Data':
                data = pag[index+1].split(' ')[0]
                #print(data)
                _data.append(data)

            if laudo[:2]!="FP":
                x='Avaliação Global'           
                if (item == x):
                    resultados=pag[index+3].split(' ')
                    #print('resultados==',resultados)

                    valor=float(resultados[-1].replace('.','').replace(',','.'))
                    valor_unit=valor/area
                    #print(valor_unit)
                    _valor_unit.append(valor_unit)
                    _valor.append(valor)
                    _ofer_trans.append(1)
            else:
                x="INFORMAÇÕES COMPLEMENTARES"
                if (item==x):
                    try:
                        i = 1
                        while i < 20:
                          #print(i,pag[index+i])
                          if pag[index+i].split(" ")[0]=="Valor":
                            break
                          i += 1

                        valor = float(pag[index+i].split(":")[1].replace('.','').replace(',','.'))
                        data = pag[index+i+1].split(":")[1]
                        #_data.append(data)
                        #print("Valor==>",valor)
                        valor_unit = valor/area
                        #print("Valor Unt==>>",valor_unit)
                        _valor_unit.append(valor_unit)
                        _valor.append(valor)
                        _ofer_trans.append(2)
                    except:
                        try:
                            valor=float(pag[index+2].split("R$")[1].split(":")[1].split("<")[0].replace('.','').replace(',','.'))
                            #data = pag[index+2].split('Data do Evento: ')[1].split("<")[0]
                            valor_unit = valor/area
                            #_data.append(data)
                            _valor_unit.append(valor_unit)
                            _valor.append(valor)
                            _ofer_trans.append(2)
                        except:
                            valor=0
                            valor_unit = 0
                            #_data.append(data)
                            _valor_unit.append(valor_unit)
                            _valor.append(valor)
                            _ofer_trans.append(2)


                
                
 

laudo: 0 7374.0747.0010171432021.01.01-LDO.pdf
laudo: 1 A-413-7374.0619.0001317772022.01.01-LDO.pdf
laudo: 2 A-413-7374.0668.0001020712022.01.01-LDO.pdf
laudo: 3 A-413-7374.1035.0010831952021.01.01-LDO.pdf
laudo: 4 A-413-7374.1089.0001058932022.01.01-LDO.pdf
laudo: 5 A-413-7374.1559.0000910042022.01.01 - LDO.pdf
laudo: 6 A-413-7374.1563.0000425822022.01.01-LDO.pdf
laudo: 7 A-413-7374.1887.0000051542022.01.01 -LDO.pdf
laudo: 8 A-413-7374.1956.0000791822022.01.01-LDO.pdf
laudo: 9 A-413-7374.1961.0010839512021.01.01-LDO.pdf
laudo: 10 A-413-7374.2002.0000669052022.01.01-LDO.pdf
laudo: 11 A-413-7374.2002.0010181102021.01.01-LDO.pdf
laudo: 12 A-413-7374.2002.0010828832021.01.01-LDO.pdf
laudo: 13 A-413-7374.2558.0000668632022.01.01-LDO.pdf
laudo: 14 A-413-7374.3418.0010232102021.01.01-LDO.pdf
laudo: 15 A-413-7374.3466.0011109822021.01.01-LDO.pdf
laudo: 16 A-413-7374.4371.0010562922021.01.01 -LDO.pdf
laudo: 17 A-413-7374.4420.0000561312022.01.01-LDO.pdf
laudo: 18 A-413-7374.4550.000005160-2022

laudo: 177 FP600022638201901010100000101.pdf
laudo: 178 FP600022895202101010100000101.pdf
laudo: 179 FP600023054201901010100000101.pdf
laudo: 180 FP600023152202101010100000101.pdf
laudo: 181 FP600023388201901010100000101.pdf
laudo: 182 FP600023389201901010100000101.pdf
laudo: 183 FP600023799201901010100000101.pdf
laudo: 184 FP600023963202101010100000101.pdf
laudo: 185 FP600024784202101010100000101.pdf
laudo: 186 FP600025538201901010100000101.pdf
laudo: 187 FP600025589201901010100000101.pdf
laudo: 188 FP600027173202101010100000101.pdf
laudo: 189 FP600029108201901010100000101.pdf
laudo: 190 FP600029933202101010100000101.pdf
laudo: 191 FP600032975201901010100000101.pdf
laudo: 192 FP600033053201901010100000101.pdf
laudo: 193 FP600036774201901010100000101.pdf
laudo: 194 FP600037052201901010100000101.pdf
laudo: 195 FP600037206201901010100000101.pdf
laudo: 196 FP600037334201901010100000101.pdf
laudo: 197 FP600037744201901010100000101.pdf
laudo: 198 FP600037857201901010100000101.pdf
laudo: 199

laudo: 359 LA000299760202101010100000101.pdf
laudo: 360 LA000311461201901010100000101.pdf
laudo: 361 LA000313492201901010100000101.pdf
laudo: 362 LA000320587201901010100000101.pdf
laudo: 363 LA000342046201901010100000101.pdf
laudo: 364 LA000342305201901010100000101.pdf
laudo: 365 LA000347864201901010100000101.pdf
laudo: 366 LA000354597201901010100000101.pdf
laudo: 367 LA000362557201901010100000102.pdf
laudo: 368 LA000380017201901010100000101.pdf
laudo: 369 LA000381080201901010100000101.pdf
laudo: 370 LA000397198201901010100000102.pdf
laudo: 371 LA000403421201901010100000101.pdf
laudo: 372 LA000412823201901010100000101.pdf
laudo: 373 LA000420380201901010100000101.pdf
laudo: 374 LA000428880201901010100000101.pdf
laudo: 375 LA000433662201901010100000101.pdf
laudo: 376 LA000439262201901010100000101.pdf
laudo: 377 LA000453142201901010100000101.pdf
laudo: 378 LA000459378201901010100000101.pdf
laudo: 379 LA000464136201901010100000101.pdf
laudo: 380 LA000465112201901010100000101 - Copia.pdf
la

laudo: 518 OS 07374.04682.0005281172021.01.01.01.pdf
laudo: 519 OS 07374.04682.0005361672021.01.01.01.pdf
laudo: 520 OS 07374.04682.0005845862021.01.01.01-LDO.pdf
laudo: 521 OS 07374.04682.0006439792021.01.01.01-LDO.pdf
laudo: 522 OS 07374.04682.0006519032020.01.01.01-LDO.pdf
laudo: 523 OS 07374.04682.0006677142021.01.01.01-LDO.pdf
laudo: 524 OS 07374.04682.0006891062021.01.01.01 -LDO.pdf
laudo: 525 OS 07374.04682.0006956422020.01.01.01-LDO.pdf
laudo: 526 OS 07374.04682.0007243232021.01.01.01-LDO.pdf
laudo: 527 OS 07374.04682.000738800-2020.01.01.01 -LDO.pdf
laudo: 528 OS 07374.04689.0006209142021.01.01.01-LDO.pdf
laudo: 529 OS 07374.04690.0002491932021.01.01.01.pdf
laudo: 530 OS 07374.04690.0005305262021.01.01.01-LDO.pdf
laudo: 531 OS 07374.04690.0006427452021.01.01.01-LDO.pdf
laudo: 532 OS 07374.07248.0001121412021.01.01.01.pdf
laudo: 533 OS 07374.07248.0001201252020.01.01.02-LDO.pdf
laudo: 534 OS 07374.07248.0005116532020.01.01.01-LDO.pdf
laudo: 535 OS 07374.07374.000037951-2021.01.

In [72]:
dict={'OS':_os,'ofer/trans':_ofer_trans,'tipo':_tipo,'andar':_andar,
      'municipio':_municipio,'end':_end,'cep':_cep,
      'idade':_idade,'vagas':_vagas,   'cons':_cons,'pad':_pad,'area':_area,
      'condomínio':_condo,      'quartos':_quartos,'banheiros':_banheiros,
      'N. equips':_equip,'valor_unit':_valor_unit,
      'valor':_valor,'data':_data,
      'lat':_lat,'long':_long,'empresa':_empresa}

df=pd.DataFrame.from_dict(dict,orient='index').transpose()
df

,OS,ofer/trans,tipo,andar,municipio,end,cep,idade,vagas,cons,...,condomínio,quartos,banheiros,N. equips,valor_unit,valor,data,lat,long,empresa
0,7374.0747.0010171432021.01.01-LDO.pdf,1,1,0,EUSEBIO,"RUA N. SRA. DO NAZARÉ, 360, Unidade 12, Residencial CONDOMÍNIO GRAND MAISON.",61.760-000,17.5,4,4,...,1,4,1,7,4189.108318,980000.0,16/11/2021,-3.878661,-38.465232,ATIVA ENGENHARIA E AVALIACOES FRANCISCO FLAVIO CARDOSO
1,A-413-7374.0619.0001317772022.01.01-LDO.pdf,1,1,0,EUSEBIO,"RUA ALAMEDA DAS ORQUIDEAS, 630, quadra 13, lote 27.",60.871-000,7.5,4,4,...,0,5,1,7,3705.971448,1150000.0,04/03/2022,-3.862742,-38.474453,ATIVA ENGENHARIA E AVALIACOES FRANCISCO FLAVIO CARDOSO
2,A-413-7374.0668.0001020712022.01.01-LDO.pdf,1,1,0,FORTALEZA,"R LOPES TROVAO, 215, Unidade 40.",60.842-230,7.5,3,4,...,1,3,2,8,2040.816327,300000.0,17/02/2022,-3.823454,-38.501365,ATIVA ENGENHARIA E AVALIACOES FRANCISCO FLAVIO CARDOSO
3,A-413-7374.1035.0010831952021.01.01-LDO.pdf,1,1,0,CAUCAIA,"R PADRE JOSE MARIA MOURA, 116, Unidade 02, CONDOMINIO RESIDENCIAL MONZA.",61.656-160,7.5,3,3,...,1,2,2,0,2596.337797,190000.0,12/12/2021,-3.767486,-38.649053,ATIVA ENGENHARIA E AVALIACOES FRANCISCO FLAVIO CARDOSO
4,A-413-7374.1089.0001058932022.01.01-LDO.pdf,1,1,0,CAUCAIA,"R MARIA CARNEIRO SILVA, 155, Unidade 01, CONDOMINIO RESIDENCIAL JÚLIO MENEZES.",61.601-640,2.5,1,4,...,1,2,2,0,2501.52532,205000.0,18/02/2022,-3.716477,-38.660259,ATIVA ENGENHARIA E AVALIACOES FRANCISCO FLAVIO CARDOSO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,OS 07374.07374.000037951-2021.01.01.01.pdf,1,2,0,ITAITINGA,"RUA MACAIBA, 440.",61.880-000,2.5,1,4,...,0,2,2,0,1650.412603,110000.0,10/05/2021,-3.907679,-38.519651,ATIVA ENGENHARIA E AVALIACOES FRANCISCO FLAVIO CARDOSO
719,OS 07888.07253.0003149392021.01.01.01.pdf,1,1,None,ITAITINGA,None,61.810-000,None,None,None,...,0,None,None,None,1574.338286,160000.0,01/10/2020,-3.913113,-38.600385,ATIVA ENGENHARIA E AVALIACOES FRANCISCO FLAVIO CARDOSO
720,OS 7374.4371.000597656-2020.01.01.01-LDO.pdf,1,1,None,PACATUBA,None,61.760-000,None,None,None,...,0,None,None,None,2158.596912,144000.0,04/09/2020,-3.861654,-38.426595,ATIVA ENGENHARIA E AVALIACOES FRANCISCO FLAVIO CARDOSO
721,OS 7374.4551.0005288172020.01.01.01-LDO.pdf,1,1,None,EUSEBIO,None,61.880-000,None,None,None,...,0,None,None,None,1862.745098,152000.0,04/12/2020,-3.92422,-38.52466,ATIVA ENGENHARIA E AVALIACOES FRANCISCO FLAVIO CARDOSO


In [73]:

df[['lat','long']]=df[['lat','long']].astype(float)

kmz=pd.read_csv('./3CEARA_RENDA2010.csv',sep=';')
kmz['lat']=kmz['lat'].str.replace(',','.')
kmz['long']=kmz['long'].str.replace(',','.')
kmz['renda']=kmz['renda'].str.replace(',','.')
kmz[['lat','long','renda']]=kmz[['lat','long','renda']].astype(float)

def closest_point(point, points):
    """ Find closest point from a list of points. """
    #return points[distance.cdist([point], points).argmin()]
    return points[cdist([point], points).argmin()]    

def match_value(df, col1, x, col2):
    """ Match value x from col1 row to value in col2. """
    return df[df[col1] == x][col2].values[0]

kmz['point'] = [(x, y) for x,y in zip(kmz['lat'], kmz['long'])]
df['point'] = [(x, y) for x,y in zip(df['lat'], df['long'])]

df['closest'] = [closest_point(x, list(kmz['point'])) for x in df['point']]
df['renda'] = [match_value(kmz, 'point', x, 'renda') for x in df['closest']]

df['renda']=df['renda'].replace(',','.').astype(float)
#df['area']=df['area'].str.replace('.','').str.replace(',','.').astype(float)

columns = ['point', 'closest']
df.drop(columns, inplace=True, axis=1)
df

,OS,ofer/trans,tipo,andar,municipio,end,cep,idade,vagas,cons,...,quartos,banheiros,N. equips,valor_unit,valor,data,lat,long,empresa,renda
0,7374.0747.0010171432021.01.01-LDO.pdf,1,1,0,EUSEBIO,"RUA N. SRA. DO NAZARÉ, 360, Unidade 12, Residencial CONDOMÍNIO GRAND MAISON.",61.760-000,17.5,4,4,...,4,1,7,4189.108318,980000.0,16/11/2021,-3.878661,-38.465232,ATIVA ENGENHARIA E AVALIACOES FRANCISCO FLAVIO CARDOSO,5667.84
1,A-413-7374.0619.0001317772022.01.01-LDO.pdf,1,1,0,EUSEBIO,"RUA ALAMEDA DAS ORQUIDEAS, 630, quadra 13, lote 27.",60.871-000,7.5,4,4,...,5,1,7,3705.971448,1150000.0,04/03/2022,-3.862742,-38.474453,ATIVA ENGENHARIA E AVALIACOES FRANCISCO FLAVIO CARDOSO,958.01
2,A-413-7374.0668.0001020712022.01.01-LDO.pdf,1,1,0,FORTALEZA,"R LOPES TROVAO, 215, Unidade 40.",60.842-230,7.5,3,4,...,3,2,8,2040.816327,300000.0,17/02/2022,-3.823454,-38.501365,ATIVA ENGENHARIA E AVALIACOES FRANCISCO FLAVIO CARDOSO,2087.75
3,A-413-7374.1035.0010831952021.01.01-LDO.pdf,1,1,0,CAUCAIA,"R PADRE JOSE MARIA MOURA, 116, Unidade 02, CONDOMINIO RESIDENCIAL MONZA.",61.656-160,7.5,3,3,...,2,2,0,2596.337797,190000.0,12/12/2021,-3.767486,-38.649053,ATIVA ENGENHARIA E AVALIACOES FRANCISCO FLAVIO CARDOSO,1288.05
4,A-413-7374.1089.0001058932022.01.01-LDO.pdf,1,1,0,CAUCAIA,"R MARIA CARNEIRO SILVA, 155, Unidade 01, CONDOMINIO RESIDENCIAL JÚLIO MENEZES.",61.601-640,2.5,1,4,...,2,2,0,2501.52532,205000.0,18/02/2022,-3.716477,-38.660259,ATIVA ENGENHARIA E AVALIACOES FRANCISCO FLAVIO CARDOSO,1313.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,OS 07374.07374.000037951-2021.01.01.01.pdf,1,2,0,ITAITINGA,"RUA MACAIBA, 440.",61.880-000,2.5,1,4,...,2,2,0,1650.412603,110000.0,10/05/2021,-3.907679,-38.519651,ATIVA ENGENHARIA E AVALIACOES FRANCISCO FLAVIO CARDOSO,817.98
719,OS 07888.07253.0003149392021.01.01.01.pdf,1,1,None,ITAITINGA,None,61.810-000,None,None,None,...,None,None,None,1574.338286,160000.0,01/10/2020,-3.913113,-38.600385,ATIVA ENGENHARIA E AVALIACOES FRANCISCO FLAVIO CARDOSO,1045.91
720,OS 7374.4371.000597656-2020.01.01.01-LDO.pdf,1,1,None,PACATUBA,None,61.760-000,None,None,None,...,None,None,None,2158.596912,144000.0,04/09/2020,-3.861654,-38.426595,ATIVA ENGENHARIA E AVALIACOES FRANCISCO FLAVIO CARDOSO,915.94
721,OS 7374.4551.0005288172020.01.01.01-LDO.pdf,1,1,None,EUSEBIO,None,61.880-000,None,None,None,...,None,None,None,1862.745098,152000.0,04/12/2020,-3.924220,-38.524660,ATIVA ENGENHARIA E AVALIACOES FRANCISCO FLAVIO CARDOSO,800.65


In [74]:

pasta=dir.split("\\")[-1]

print(pasta)
df.to_excel('./resultados/'+pasta+'.xlsx',index=False)


2019-2020-2021
